In [6]:
!pip install sqlmodel==0.0.22 -q
!pip install beautifulsoup4==4.12.0 -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [11]:
import requests
import sqlmodel
from bs4 import BeautifulSoup as bs

In [4]:
resp = requests.get("https://www.wikiart.org/en/artists-by-art-institution/")

In [5]:
with open('institutions.html', 'w') as f:
    f.write(resp.text)

In [ ]:
class ArtistBase(sqlmodel.SQLModel):
    name: str = sqlmodel.Field(primary_key=True, default=None)
    art_movement: list[str]
    school_and_groups: list[str] | None
    genres: list[str] | None
    fields: list[str] | None
    nationalities: list[str] | None
    art_institutions: list[str] | None
    wikipedia_article: str | None
    article: str | None
    references: list[str] | None


class Artist(ArtistBase, table=True):
    artworks: list["Artwork"] = sqlmodel.Relationship(back_populates="artist")


class ArtworkBase(sqlmodel.SQLModel):
    name: str = sqlmodel.Field(primary_key=True, default=None)
    styles: list[str] | None
    genre: list[str] | None
    media: list[str] | None


class Artwork(ArtworkBase, table=True):
    artist: Artist = sqlmodel.Relationship(back_populates="artworks")

ValueError: <class 'list'> has no matching SQLAlchemy type

In [ ]:
def get_artists_by_movement(movement: str):
    pass


def get_artists_by_school_or_group(school_or_group: str):
    pass


def get_artists_by_genre(genre: str):
    pass


def get_artists_by_field(field: str):
    pass


def get_artist_by_nationality(nationality: str):
    pass


def get_institutions() -> list[str]:
    """
    Returns a list of institutions
    https://www.wikiart.org/en/artists-by-art-institution
    """
    base_url = "https://www.wikiart.org/en/artists-by-art-institution/"
    resp = requests.get(base_url)
    soup = bs(resp.text, 'html.parser')
    bullets = soup.find_all('ul', {"class": "dictionaries-list", "ng-if": "!tabChanged"})
    if len(bullets) != 1:
        raise ValueError("More than one list of institutions found")

    institutions = [li.text for li in bullets[0].find_all('li')]
    return institutions


def get_artist_by_institution():
    base_url = "https://www.wikiart.org/en/artists-by-art-institution/"
    resp = requests.get(base_url)
    soup = bs(resp.text, 'html.parser')

    bullets = soup.find_all('ul', {"class": "dictionaries-list", "ng-if": "!tabChanged"})

    if len(bullets) != 1:
        print("ruh roh")

    institutions = [li.text for li in bullets[0].find_all('li')]
    return institutions

In [18]:
get_artist_by_institution()

['\nAustralia\n',
 '\nSouth Australian School of Design, Adelaide, Australia 3\n',
 '\nNational Gallery of Victoria Art School, Melbourne, Victoria, Australia 7\n',
 '\nAustria\n',
 '\nUniversität für angewandte Kunst Wien (Kunstgewerbeschule), Vienna, Austria 4\n',
 '\nAkademie der Bildenden Künste Wien, Vienna, Austria 23\n',
 '\nBelarus\n',
 '\nVitebsk Art School, Vitebsk, Belarus 4\n',
 '\nBelgium\n',
 '\nAcadémie Royale des Beaux-Arts (ARBA-ESA), Brussels, Belgium 8\n',
 '\nRoyal Academy of Fine Arts, Antwerp, Belgium 12\n',
 '\nDenmark\n',
 '\nRoyal Danish Academy of Fine Arts, Copenhagen, Denmark 9\n',
 '\nFrance\n',
 '\nAcadémie des Beaux-Arts, Paris, France 20\n',
 '\nAcadémie Julian, Paris, France 58\n',
 '\nÉcole des Beaux-Arts, Paris, France 87\n',
 '\nAcadémie Royale de Peinture et de Sculpture, Paris, France 8\n',
 '\nAcadémie Colarossi, Paris, France 22\n',
 '\nAcadémie de La Palette, Paris, France 12\n',
 '\nAcadémie de la Grande Chaumière, Paris, France 10\n',
 '\nGeor